## 임베딩 레이어 이해하기

In [ ]:
import torch
import torch.nn as nn

vocab = {'cat' : 0, 'dog' : 1, 'fish' : 2}
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 100
embedding = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
embedding_vector = embedding(torch.LongTensor([vocab['fish']]))
print(embedding_vector)
print(embedding_vector.shape)

tensor([[-0.4092, -3.1752,  0.1187, -0.5941,  0.5692,  0.4907, -0.0151,  0.2598,
         -1.0869,  0.2124, -0.4289,  1.6072, -0.9780, -0.0465, -0.9104, -0.7676,
         -0.3943,  0.0172, -0.5581,  1.1185, -1.1009, -1.6278,  0.4446,  2.4512,
         -1.0693,  0.6192, -0.7371, -0.9022,  0.9065,  0.2275,  0.5507, -2.6993,
          0.7755,  0.5312, -0.5181, -0.6739, -0.8574, -1.0131,  1.9634,  1.7328,
         -0.1638,  0.6241, -0.1056, -0.5967,  0.6524, -0.2914, -0.2250,  0.4219,
          0.7782, -1.8572,  0.6273, -1.0137,  1.0835, -0.6329, -0.6966,  0.8957,
         -1.4627,  0.4025, -1.6189,  0.3923,  0.3819, -1.6502,  0.5439, -1.4938,
          0.3813,  1.9770, -1.2082,  1.5503, -0.3824, -0.5267, -1.1660, -0.9868,
         -0.3049,  1.8171, -1.3952,  0.2935,  1.2565,  1.5880,  0.0251,  0.3749,
          0.4500, -0.9892, -0.7103, -1.2616, -1.3352, -0.0684, -1.3923, -1.0886,
         -0.1699, -1.8787,  2.9153, -0.6274, -0.2041, -1.0792,  0.1053, -0.2104,
          1.6592, -0.3600, -

## 모델링

In [1]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 26.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
from konlpy.tag import Okt
import re
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt', sep = '\t')
df = df.dropna()
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [3]:
okt = Okt()

def preprocessing_text(text):
    # 한글, 영어, 숫자, 공백, ?!.,을 제외한 나머지 문자 제거
    result_text = re.sub('[^ ?,.!A-Za-z0-9가-힣+]', ' ', text)
    result_text = okt.morphs(result_text)
    return result_text

In [ ]:
#### 실제로 실행시키실 때 활용
# df['tokenized'] = df['document'].apply(preprocessing_text)
# token_list = df['tokenized'].to_list()

# # 전처리한 데이터 저장
# with open('tokenized_nsmc.pkl', 'wb') as file:
#     pickle.dump(token_list, file)
# df['tokenized']  = token_list

In [4]:
import pickle
with open('tokenized_nsmc.pkl', 'rb') as file:
    token_list = pickle.load(file)
df['tokenized']  = token_list

In [5]:
from collections import Counter

UNKNOWN_TOKEN = "<unk>"

# 단어 사전 구축
def build_vocab(token_list):
    # 모든 토큰을 모아서 빈도 계산
    counter = Counter(token for tokens in token_list for token in tokens)
    # 단어 사전 생성 (UNKNOWN_TOKEN 포함)
    vocab = {UNKNOWN_TOKEN: 0}
    vocab.update({token: idx for idx, (token, _) in enumerate(counter.items(), start=1)})
    return vocab

# 사전 생성
token_list = df['tokenized']  # token_list는 DataFrame에서 토큰화된 열
vocab = build_vocab(token_list)

# 사전에 없는 단어는 UNKNOWN_TOKEN으로 처리
def text_pipeline(x):
    return [vocab.get(token, vocab[UNKNOWN_TOKEN]) for token in x]

# DataFrame에 숫자 시퀀스 추가
df['encoded'] = df['tokenized'].apply(text_pipeline)

In [8]:
# 훈련/테스트 데이터셋 분리
train_data, test_data = train_test_split(df, test_size=0.3, random_state=0, shuffle = True)

In [9]:
class MovieReviewDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data.iloc[idx]['encoded']), torch.tensor(self.data.iloc[idx]['label'])

train_dataset = MovieReviewDataset(train_data)
test_dataset = MovieReviewDataset(test_data)

In [10]:
# 배치단위로 데이터를 묶기
def collate_fn(batch):
    # 텍스트 데이터와 레이블 데이터를 분리
    text_data = [item[0] for item in batch]
    label_data = [item[1] for item in batch]

    # 가장 긴 사이즈 기준으로 텍스트 데이터만 패딩 처리
    # batch_first=True 배치 의 위치 (배치 크기, 시퀀스 길이, 특성 수)로 구성
    padded_text_data = pad_sequence(text_data, batch_first=True)

    # 레이블 데이터는 텐서로 변환
    label_data = torch.tensor(label_data)

    return padded_text_data.to(device), label_data.to(device)

# collate_fn : 데이터를 넘겨주기 전에 사용할 함수
# batch_first = True -> 배치를 첫번째 차원으로 (batch_size, sequence_length, features)
train_loader = DataLoader(train_dataset, batch_size=32, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)

In [19]:
class ReviewClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(ReviewClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x.long())
        embedded = embedded.mean(dim=1)
        output = self.fc(embedded)
        output = self.sigmoid(output)
        return output

In [22]:
import torch.optim as optim
# 하이퍼파라미터 설정
max_length = 100  # 임의로 설정한 문장 최대 길이
vocab_size = len(vocab)  # 단어 집합 크기
embedding_dim = 128  # 임베딩 차원

# 모델 초기화
model = ReviewClassifier(vocab_size, embedding_dim).to(device)

# 손실 함수 및 옵티마이저
criterion = nn.BCELoss() # Binary Cross Entropy
optimizer = optim.Adam(model.parameters())

In [23]:
# 학습
for epoch in range(5):  # 에포크 수
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        # squeeze() : 크기가 1인 차원 제거
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{5}, Loss: {loss.item()}')
# state_dict() : weight 저장
model = torch.save(model.state_dict(), 'ReviewClassifier.pt') # save model

KeyboardInterrupt: 

In [26]:
# 모델 정의 (동일한 구조를 가진 모델 필요)
max_length = 100  # 임의로 설정한 문장 최대 길이
vocab_size = len(vocab)  # 단어 집합 크기
embedding_dim = 128  # 임베딩 차원
loaded_model = ReviewClassifier(vocab_size, embedding_dim)

# 저장된 모델 불러오기
loaded_model.load_state_dict(torch.load('ReviewClassifier.pt', weights_only=True))
loaded_model.eval()  # 평가 모드로 설정

ReviewClassifier(
  (embedding): Embedding(119910, 128)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [28]:
def predict(model, sentence, text_pipeline):
    loaded_model.eval()
    with torch.no_grad():
        tokens = text_pipeline(sentence)  # 텍스트 전처리
        text_tensor = torch.tensor(tokens).unsqueeze(0)  # 배치 차원 추가
        output = model(text_tensor)
        if output.item() >= 0.5:
          return 'positive'
        else:
          return 'negative'

dic = {0: 'negative', 1: 'positive'}

# 예시 텍스트
idx = 10023
example_text = train_data['document'].iloc[idx]
correct = train_data['label'].iloc[idx]
# 예측 수행
prediction = predict(loaded_model, example_text, text_pipeline)
print(example_text)
print(f'Predict: {prediction}\nCorrect :{dic[correct]}')

평점에 낚인 영화... 억지 설정, 억지 웃음. 과장된 연기... 괜히 시간만 낭비했음.
Predict: positive
Correct :negative
